# Информационный поиск

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://nlp.stanford.edu/~manning/xyzzy/Intro_Inform_Retrieval_Russian.pdf
* https://en.wikipedia.org/wiki/Jaccard_index
* https://en.wikipedia.org/wiki/TF-IDF
* https://en.wikipedia.org/wiki/Okapi_BM25

## Задачи для совместного разбора

1\. Дан корпус текстов. Построить прямой и обратный индексы для слов из текста

In [ ]:
corpus = [
    "Первым специальным индексом для запросов с джокером общего вида является перестановочный индекс",
    "Методы усовершенствования индексов для расширения функциональных возможностей и повышения скорости поиска"
]

In [ ]:
!pip install pymorphy2

In [ ]:
from nltk import word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [ ]:
index = {}
for idx, doc in enumerate(corpus):
  index[idx] = [
    morph.parse(w)[0].normal_form
    for w in word_tokenize(doc)
  ]

In [ ]:
index

In [ ]:
from collections import defaultdict
# inv_index = defaultdict(list)
inv_index = defaultdict(set)

# defaultdict(f)
# if k not in d:
# d[k] = f() # list()

for idx, tokens in index.items():
  for tok in tokens:
    inv_index[tok].add(idx)

2\. Посчитать индекс Жаккара для предложений из заданного корпуса.

In [ ]:
index[0]

['первый',
 'специальный',
 'индекс',
 'для',
 'запрос',
 'с',
 'джокер',
 'общий',
 'вид',
 'являться',
 'перестановочный',
 'индекс']

In [ ]:
index[1]

['метод',
 'усовершенствование',
 'индекс',
 'для',
 'расширение',
 'функциональный',
 'возможность',
 'и',
 'повышение',
 'скорость',
 'поиск']

In [ ]:
(
    len(set(index[0]).intersection(set(index[1]))),
    len(set(index[0]).union(set(index[1])))
)

(2, 20)

## Задачи для самостоятельного решения

In [105]:
import json
import nltk
from nltk import word_tokenize
import pymorphy2
from collections import defaultdict
from collections import Counter
from math import log
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ma2si\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<p class="task" id="1"></p>

### 1
Считайте тексты новостей из файла `news.json`. Назовём прямым индексом словарь, где ключами являются номера новостей, а значениями - списки нормализованных слов, входящих в эту новость. Номер новости определяется ее положением в файле. Назовём обратным индексом словарь, где ключами являются нормальные формы слов, а значениями - множества номеров новостей, которые содержат данное слово (не обязательно в нормальной форме). Постройте прямой и обратный индекс. Выведите из длины на экран. Выведите значения обратного индекса по ключу "москва".

- [x] Проверено на семинаре

In [133]:
with open('data/news.json', 'r', encoding='utf8') as f:
    corpus = json.load(f)

In [134]:
print(corpus)

['российский бюджет в марте недополучил более 300 миллиардов рублей нефтегазовых доходов', 'банк россии решил снизить ключевую ставку с 20  до 17 ', 'мыс идокопас  в нато назвали первую цель для начала полномасштабного вторжения в россию ', 'минобороны рф заявило об ударах ракетами «калибр» по запорожскому алюминиевому комбинату и уничтожении ангаров с иностранным оружием', 'госдеп попросил россию исключить санта клауса из санкционного списка на время рождественских праздников ', 'я ее не трогал распустивший язык пригожин отпихнул от себя оскандалившуюся вайкуле', 'приложение «аэрофлота» больше недоступно в app store', 'виталий милонов награждён высшим венгерским орденом за борьбу с гомосексуализмом ', '«яндекс еду» наказали за утечку адресов  телефонов и имен десятков тысяч клиентов  суд назначил минимально возможный штраф — 60 тысяч рублей', 'трамп предложил перенести новые санкции против россии из за дня народного единства ', 'афганистан  присоединение кабула к санкциям против москв

In [135]:
morph = pymorphy2.MorphAnalyzer()

In [136]:
index = {}
for idx, doc in enumerate(corpus):
  index[idx] = [
    morph.parse(w)[0].normal_form
    for w in word_tokenize(doc)
  ]
print(index)

{0: ['российский', 'бюджет', 'в', 'март', 'недополучить', 'более', '300', 'миллиард', 'рубль', 'нефтегазовый', 'доход'], 1: ['банк', 'россия', 'решить', 'снизить', 'ключевой', 'ставка', 'с', '20', 'до', '17'], 2: ['мыс', 'идокопас', 'в', 'нато', 'назвать', 'первый', 'цель', 'для', 'начало', 'полномасштабный', 'вторжение', 'в', 'россия'], 3: ['минобороны', 'рф', 'заявить', 'о', 'удар', 'ракета', '«', 'калибр', '»', 'по', 'запорожский', 'алюминиевый', 'комбинат', 'и', 'уничтожение', 'ангар', 'с', 'иностранный', 'оружие'], 4: ['госдеп', 'попросить', 'россия', 'исключить', 'сант', 'клаус', 'из', 'санкционный', 'список', 'на', 'время', 'рождественский', 'праздник'], 5: ['я', 'она', 'не', 'трогать', 'распустить', 'язык', 'пригожин', 'отпихнуть', 'от', 'себя', 'оскандалиться', 'вайкуль'], 6: ['приложение', '«', 'аэрофлот', '»', 'большой', 'недоступный', 'в', 'app', 'store'], 7: ['виталий', 'милон', 'наградить', 'высокий', 'венгерский', 'орден', 'за', 'борьба', 'с', 'гомосексуализм'], 8: ['«',

In [137]:
len(index)

500

In [138]:
inv_index = defaultdict(list)

for idx, tokens in index.items():
  for tok in tokens:
    inv_index[tok].append(idx)

print(inv_index)

defaultdict(<class 'list'>, {'российский': [0, 10, 17, 35, 43, 48, 71, 81, 101, 104, 114, 122, 125, 129, 135, 151, 160, 167, 216, 219, 229, 254, 259, 266, 300, 301, 396, 399, 437, 442, 451, 456, 472], 'бюджет': [0, 204], 'в': [0, 2, 2, 6, 11, 16, 20, 20, 24, 28, 28, 30, 31, 32, 33, 34, 35, 37, 40, 41, 42, 50, 51, 52, 53, 58, 60, 63, 64, 66, 68, 70, 70, 72, 74, 77, 81, 82, 84, 86, 87, 88, 89, 91, 92, 93, 95, 96, 97, 103, 103, 108, 113, 113, 113, 116, 117, 118, 123, 123, 127, 130, 132, 133, 134, 135, 136, 136, 141, 148, 149, 153, 153, 154, 158, 158, 159, 161, 166, 168, 170, 171, 171, 173, 175, 176, 177, 178, 178, 178, 179, 182, 188, 188, 189, 189, 192, 199, 200, 201, 203, 205, 206, 213, 214, 215, 217, 218, 219, 221, 224, 225, 228, 229, 229, 230, 232, 234, 236, 238, 241, 243, 245, 246, 248, 249, 257, 258, 258, 259, 261, 261, 264, 265, 265, 266, 269, 273, 274, 276, 277, 278, 280, 282, 283, 284, 288, 289, 290, 292, 297, 300, 303, 304, 305, 307, 308, 309, 309, 313, 316, 318, 320, 324, 325, 3

In [139]:
len(inv_index)

2443

In [140]:
print(inv_index['москва'])

[10, 28, 32, 88, 111, 134, 148, 176, 182, 190, 199, 205, 207, 225, 240, 275, 280, 297, 324, 350, 374, 416, 466, 486]


<p class="task" id="2"></p>

### 2
Для каждого документа из текста определите, как часто слова этого документа встречаются в заданном тексте, воспользовавшись обратным индексом. Выведите на экран текст топ-3 новостей, наиболее похожих по набору токенов на заданный текст. Выведите на экран текст найденных новостей и количество совпадающих токенов.

- [x] Проверено на семинаре

In [141]:
example = "Жириновский предложил перенести столицу из Москвы"

In [142]:
example_tokens = []
for token in word_tokenize(example):
    example_tokens.append(morph.parse(token)[0].normal_form)
example_tokens

['жириновский', 'предложить', 'перенести', 'столица', 'из', 'москва']

In [144]:
sents = []
for token in example_tokens:
    sents.extend(inv_index[token])

num_counts = Counter(sents)

sorted_counts = sorted(num_counts.items(), key=lambda x: x[1], reverse=True)

for num, count in sorted_counts[:3]:
    print(f"Новость {num} содержит {count} слова из example {index[num]}\nОбщие слова:  {set(index[num]).intersection(example_tokens)}\n")

Новость 9 содержит 3 слова из example ['трамп', 'предложить', 'перенести', 'новый', 'санкция', 'против', 'россия', 'из', 'за', 'день', 'народный', 'единство']
Общие слова:  {'предложить', 'перенести', 'из'}

Новость 78 содержит 2 слова из example ['31', 'декабрь', 'владимир', 'жириновский', 'предложить', 'имя', 'свой', 'преемник']
Общие слова:  {'предложить', 'жириновский'}

Новость 112 содержит 2 слова из example ['жириновский', 'предложить', 'дать', 'ремень', 'создатель', 'гачимучить', 'кавер', 'на', 'известный', 'песня']
Общие слова:  {'предложить', 'жириновский'}



<p class="task" id="3"></p>

### 3
Для заданного текста новости найдите топ-3 наиболее похожих новости, воспользовавшись коэффициентом Жаккара. Для определения количества общих слов в тексте используйте обратный индекс.  Выведите на экран текст найденных новостей и значение коэффициента Жаккара для каждой новости.

In [145]:
example = "Жириновский предложил перенести столицу из Москвы"

In [146]:
example_tokens = []
for token in word_tokenize(example):
    example_tokens.append(morph.parse(token)[0].normal_form)
example_tokens = set(example_tokens)    
example_tokens

{'жириновский', 'из', 'москва', 'перенести', 'предложить', 'столица'}

In [147]:
jaccard_scores = []

for i, text in index.items():
    text = set(text)
    intersection = len(example_tokens.intersection(text))
    union = len(example_tokens.union(text))
    jaccard_coefficient = intersection / union
    jaccard_scores.append((i, jaccard_coefficient))

jaccard_scores.sort(key=lambda x: x[1], reverse=True)
top_3_similar_news = jaccard_scores[:3]

for i, score in jaccard_scores[:3]:
    print(f"Новость {i} J = {score} {index[i]}\n")

Новость 9 J = 0.2 ['трамп', 'предложить', 'перенести', 'новый', 'санкция', 'против', 'россия', 'из', 'за', 'день', 'народный', 'единство']

Новость 36 J = 0.18181818181818182 ['международный', 'универсиада', '2023', 'год', 'перенести', 'из', 'екатеринбург']

Новость 270 J = 0.18181818181818182 ['токай', 'предложить', 'исключить', 'россия', 'из', 'состав', 'снг']



<p class="task" id="4"></p>

### 4
Реализуйте функцию для расчета TF-IDF.
![image.png](attachment:image.png)
![image-2.png](attachment:image-2.png)
![image-3.png](attachment:image-3.png)

Для расчета обратной частоты документа воспользуйтесь обратным индексом из предыдущей задачи. При расчете IDF добавьте сглаживание, прибавив единицу к числителю и знаменателю дроби.

In [148]:
def calculate_tfidf(word, text, index, inv_index):

    tf = (text.count(word)) / len(text)
    
    idf = log((len(index) + 1) / (len(inv_index[word])) + 1)
    
    tfidf = tf * idf
    
    return tfidf

In [149]:
tfidf = calculate_tfidf('google', index[480], index, inv_index)
print(f"TF-IDF слова 'google' в новости 480: {tfidf}")

TF-IDF слова 'google' в новости 480: 0.36599742710023275


<p class="task" id="5"></p>

### 5
Используя собственную реализацию TD-IDF, закодируйте новости. Результатом должна являться матрица `MxN`, где `M` - количество новостей в корпусе, `N` - размер обратного индекса. Выведите форму полученной матрицы на экран.

In [161]:
tfidf_matrix = []

for doc_id, tokens in index.items():
    tfidf_vector = []

    for word in inv_index.keys():
        tfidf_score = calculate_tfidf(word, index[doc_id], index, inv_index)
        tfidf_vector.append(tfidf_score)
        
    tfidf_matrix.append(tfidf_vector)

In [162]:
np.array(tfidf_matrix).shape

(500, 2443)

<p class="task" id="6"></p>

### 6
Для заданного текста новости найдите топ-3 наиболее похожих новости, воспользовавшись матрицей TD-IDF из предыдущего задания. Для определения схожести используйте функцию косинусного расстояния. Выведите на экран текст найденных новостей и значение метрики близости для каждой новости.

In [153]:
example = "Жириновский предложил перенести столицу из Москвы"

In [154]:
example_tokens = []
for token in word_tokenize(example):
    example_tokens.append(morph.parse(token)[0].normal_form)  
example_tokens

['жириновский', 'предложить', 'перенести', 'столица', 'из', 'москва']

In [155]:
example_tfidf = []

for word in inv_index.keys():
    tfidf_score = calculate_tfidf(word, example_tokens, index, inv_index)
    example_tfidf.append(tfidf_score)

len(example_tfidf)

2443

In [171]:
cosine_similarities = cosine_similarity([example_tfidf], tfidf_matrix)

cosine_similarities = cosine_similarities[0]

top_indices = np.argsort(cosine_similarities)[-3:][::-1]

for idx in top_indices:
    similarity_value = cosine_similarities[idx]
    vector = tfidf_matrix[idx]
    print(f"Новость номер {idx} : '{corpus[idx]}', косинусное сходство: {similarity_value}\n")

Новость номер 9 : 'трамп предложил перенести новые санкции против россии из за дня народного единства ', косинусное сходство: 0.2791459281886526

Новость номер 78 : '31 декабря владимир жириновский предложит имя своего преемника ', косинусное сходство: 0.23237550223962905

Новость номер 330 : 'в госдуме зарегистрирован законопроект о переносе столицы россии в великий новгород ', косинусное сходство: 0.22909198817042312



<p class="task" id="7"></p>

### 7
Для заданного текста новости найдите топ-3 наиболее похожих новости, воспользовавшись собственной реализацией функции ранжирования [BM25](https://en.wikipedia.org/wiki/Okapi_BM25). Выведите на экран текст найденных новостей и значение метрики ранжирования для каждой новости.

In [ ]:
example = "Жириновский предложил перенести столицу из Москвы"

## Обратная связь
- [ ] Хочу получить обратную связь по решению